<a href="https://colab.research.google.com/github/PriyalManiar/wildfire_forecasting/blob/main/Climate_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import pandas as pd
import os
from google.colab import files
import numpy as np
import pandas as pd


# List of CSV filenames (replace with your actual filenames or use os.listdir)
csv_files = [
    '/content/4011578.csv',
    '/content/4011579.csv',
    '/content/4011580.csv',
    '/content/4011582.csv',
    '/content/4011594.csv',
    '/content/4011595.csv',
    '/content/4011667_ramon.csv',
    '/content/4011670_ramon2.csv'
]

df_combined = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# Save the merged file
merged_filename = 'merged_climate.csv'
df_combined.to_csv(merged_filename, index=False)

# # Download the file to your system
# files.download(merged_filename)

# Now read it into a DataFrame again (this step is optional if df_combined is already in memory)
df = pd.read_csv(merged_filename)

<ipython-input-81-82ac5c713831>:20: DtypeWarning: Columns (3,21,27,28,29,30,31,32,33,43,44,49,50,52,54,70,72,89,96) have mixed types. Specify dtype option on import or set low_memory=False.
  df_combined = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
<ipython-input-81-82ac5c713831>:20: DtypeWarning: Columns (44,53,72,89) have mixed types. Specify dtype option on import or set low_memory=False.
  df_combined = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
<ipython-input-81-82ac5c713831>:20: DtypeWarning: Columns (3,21,27,28,29,30,31,32,33,34,37,38,43,44,45,49,50,54,56,57,59,60,61,62,63,64,70,72,73,89,96,97,98,99,100,101,102,103,104,105,106,107,108,115,118,120) have mixed types. Specify dtype option on import or set low_memory=False.
  df_combined = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
<ipython-input-81-82ac5c713831>:20: DtypeWarning: Columns (31,37,38,49,54,56,59,60,62,64,115,118,120) have mixed type

In [82]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['YearMonth'] = df['DATE'].dt.to_period('M')

df_selected = df[['STATION', 'DATE', 'BackupLatitude', 'BackupLongitude', 'YearMonth',
                  'BackupElevation', 'DailyAverageWindSpeed', 'DailyAverageRelativeHumidity',
                  'DailyAverageDryBulbTemperature', 'DailyPeakWindDirection']].copy()

In [83]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652169 entries, 0 to 652168
Data columns (total 10 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   STATION                         652169 non-null  int64         
 1   DATE                            652169 non-null  datetime64[ns]
 2   BackupLatitude                  169471 non-null  float64       
 3   BackupLongitude                 169471 non-null  float64       
 4   YearMonth                       652169 non-null  period[M]     
 5   BackupElevation                 169471 non-null  float64       
 6   DailyAverageWindSpeed           17734 non-null   float64       
 7   DailyAverageRelativeHumidity    4567 non-null    float64       
 8   DailyAverageDryBulbTemperature  17533 non-null   object        
 9   DailyPeakWindDirection          17975 non-null   object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(2), p

In [84]:
df_selected['DailyPeakWindDirection'] = pd.to_numeric(df_selected['DailyPeakWindDirection'], errors='coerce')
df_selected['DailyPeakWindDirection'] = df_selected['DailyPeakWindDirection'].astype(float)

In [85]:
df_selected['DailyAverageDryBulbTemperature'] = pd.to_numeric(df_selected['DailyAverageDryBulbTemperature'], errors='coerce')
df_selected['DailyAverageDryBulbTemperature'] = df_selected['DailyAverageDryBulbTemperature'].astype(float)

In [86]:
# Find all instances where the value is exactly '60s'
df_selected[df_selected['DailyAverageDryBulbTemperature'] == '60s']

,STATION,DATE,BackupLatitude,BackupLongitude,YearMonth,BackupElevation,DailyAverageWindSpeed,DailyAverageRelativeHumidity,DailyAverageDryBulbTemperature,DailyPeakWindDirection


In [87]:
monthly_averages = df_selected.groupby(['STATION', 'YearMonth'], as_index=False).agg({
    'BackupLatitude': 'first',
    'BackupLongitude': 'first',
    'DailyAverageWindSpeed': 'mean',
    'DailyAverageRelativeHumidity': 'mean',
    'DailyAverageDryBulbTemperature': 'mean',
    'DailyPeakWindDirection': 'mean',
    'BackupElevation': 'first'
})


monthly_averages.rename(columns={
    'DailyAverageWindSpeed': 'MonthlyAverageWindSpeed',
    'DailyAverageRelativeHumidity': 'MonthlyAverageRelativeHumidity',
    'DailyAverageDryBulbTemperature': 'MonthlyAverageTemperature',
    'DailyPeakWindDirection': 'MonthlyAverageWindDirection',
    'BackupLatitude': 'Latitude',
    'BackupLongitude': 'Longitude',
    'YearMonth': 'Date'
}, inplace=True)


In [88]:
monthly_averages.head(5)

,STATION,Date,Latitude,Longitude,MonthlyAverageWindSpeed,MonthlyAverageRelativeHumidity,MonthlyAverageTemperature,MonthlyAverageWindDirection,BackupElevation
0,72290023188,2012-01,32.7336,-117.1831,3.264516,62.161290,58.419355,256.129032,15.0
1,72290023188,2012-02,32.7336,-117.1831,5.272414,62.862069,58.103448,265.862069,15.0
2,72290023188,2012-03,32.7336,-117.1831,6.280645,63.225806,58.419355,270.000000,15.0
3,72290023188,2012-04,32.7336,-117.1831,6.310000,67.400000,61.400000,265.000000,15.0
4,72290023188,2012-05,32.7336,-117.1831,6.654839,72.451613,63.967742,253.870968,15.0


In [89]:
def circular_mean(degrees):
    radians = np.deg2rad(degrees.dropna())
    sin_mean = np.mean(np.sin(radians))
    cos_mean = np.mean(np.cos(radians))
    mean_angle = np.arctan2(sin_mean, cos_mean)
    mean_angle_deg = np.rad2deg(mean_angle)
    return (mean_angle_deg + 360) % 360  # Ensure result is in [0, 360)

In [90]:
monthly_averages = monthly_averages.round(3)

In [91]:
monthly_averages.to_csv('monthly_averages.csv', index=False)
files.download('monthly_averages.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>